In [7]:
!pip install spacy-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [8]:
import glob
import os
import pandas as pd
import re
import json

orig_text_dir = "/data"
annotate_text_dir = "/annotations"

files = glob.glob(os.path.join(orig_text_dir, "*.txt"))
ner_annotation = []

for file in files:
    base = os.path.basename(file)
    annotations = os.path.join(annotate_text_dir, base)
    annotations = pd.read_csv(annotations, sep='\t')
    with open(file, "r") as f:
        full_txt = f.read()
    chunks, chunk_size = len(full_txt), 256
    text = [full_txt[i:i + chunk_size] for i in range(0, chunks, chunk_size)]

    for txt in text:
        for k, col in enumerate(annotations.columns):
            if col.lower().strip() == 'tag':
                col_tag = col
            if col.lower().strip() == 'value':
                col_value = col

        entities = []
        st_covered = []
        end_covered = []

        for i in range(annotations.shape[0]):
            try:
                value = annotations[col_value].iloc[i].strip()
                value_escaped = re.escape(value)  # Escape the value for regex
                occurrences = [m.span() for m in re.finditer(value_escaped, txt)]
                tag = annotations[col_tag].iloc[i]
                for occurance in occurrences:
                    st = occurance[0]
                    end = occurance[1]
                    if st not in st_covered and end not in end_covered:
                        st_covered.append(st)
                        end_covered.append(end)
                        entities.append((st, end, tag))
            except Exception as ex:
                print(f"Exception occurred for value '{value}': {ex}")
                continue

        final_entities = []
        for entity in entities:
            st = entity[0]
            end = entity[1]
            overlap = False
            for entity_ in entities:
                st_ = entity_[0]
                end_ = entity_[1]
                if st_ < st and end_ > end:
                    overlap = True
            if not overlap:
                final_entities.append(entity)
        ner_annotation.append((txt, {'entities': final_entities}))

# Optionally, save the ner_annotation for later use
with open('ner_annotation.json', 'w') as f:
    json.dump(ner_annotation, f, indent=4)


In [9]:
ner_annotation

[('\nRecord 175:\n\n\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 47-year-old female who returns to the clinic for a followup visit.  The patient has multiple medical problems including severe and extreme hypertension as well as high blood pressure and chronic',
  {'entities': []}),
 (' renal failure for which she is currently attending dialysis sessions 3 times per week.  The patient also is currently being treated for hot flashes with Prempro.  The patient states that she ran out of her blood pressure medications including the patch 2 ',
  {'entities': []}),
 ('days ago.  She also complains of frequent popping and loss of hearing in her right ear.\n\n\n\nPHYSICAL EXAMINATION:  A middle-aged female in no acute distress.  Blood pressure is 182/94, weight is 211 pounds, and pulse is 76.  Chest is clear bilaterally.  Hea',
  {'entities': []}),
 ('rt, regular rate and rhythm, positive S1, S2.  Negative S3, S4, no murmur.  Abdomen is soft, nontender, positive bowel sounds.  Lower extre

In [10]:
import random
import json

random.shuffle(ner_annotation)
train_index = int(0.7 * len(ner_annotation))
valid_index = int(0.85 * len(ner_annotation))

TRAIN_DATA = ner_annotation[:train_index]
VALID_DATA = ner_annotation[train_index:valid_index]
TEST_DATA = ner_annotation[valid_index:]

# Save the data to JSON files for use with SpaCy
def save_to_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

save_to_json(TRAIN_DATA, "train_data.json")
save_to_json(VALID_DATA, "valid_data.json")
save_to_json(TEST_DATA, "test_data.json")


In [11]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

def convert_to_docbin(data, output_file):
    nlp = spacy.blank("en")  # load a new spacy model
    db = DocBin()  # create a DocBin object

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)  # create doc object from text
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents  # label the text with the ents
        db.add(doc)

    db.to_disk(output_file)  # save the docbin object

convert_to_docbin(TRAIN_DATA, "train.spacy")
convert_to_docbin(VALID_DATA, "valid.spacy")
convert_to_docbin(TEST_DATA, "test.spacy")


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
  4%|▍         | 150/3477 [00:00<00:06, 508.57it/s]

Skipping entity
Skipping entity
Skipping entity


  8%|▊         | 269/3477 [00:00<00:05, 538.01it/s]

Skipping entity
Skipping entity
Skipping entity


 11%|█▏        | 398/3477 [00:00<00:05, 597.67it/s]

...

Skipping entity
Skipping entity
Skipping entity
Skipping entity


 71%|███████▏  | 532/746 [00:00<00:00, 729.15it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 746/746 [00:01<00:00, 707.71it/s]


Skipping entity
Skipping entity
Skipping entity


In [12]:
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
!python -m spacy debug config config.cfg


============================= Config validation =============================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [14]:
!python -m spacy train config.cfg --output ./output --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of BertModel were not initialized from the model checkpoint at bert_model_data 1 and a

In [16]:
!python -m spacy evaluate ./output/model-best ./test.spacy --gpu-id 0


ℹ Using GPU: 0
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK     100.00
NER P   57.93 
NER R   44.92 
NER F   50.60 
SPEED   6124  


=============================== NER (per type) ===============================

             